In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
import lightgbm as lgb
import pickle
import os

In [2]:
flight_data = (
    pd.read_csv('final_dataset_cleaned_v1.csv')
)

In [3]:
flight_data.shape

(12658696, 29)

In [4]:
flight_data.head(n = 3)

YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK     FL_DATE TAIL_NUM  \
0  2020        1      1             1            3  2020-01-01   N951WN   
1  2020        1      1             1            3  2020-01-01   N467WN   
2  2020        1      1             1            3  2020-01-01   N7885A   

  ORIGIN ORIGIN_CITY_NAME DEST  ...                 time  temp  dwpt  rhum  \
0    ONT      Ontario, CA  SFO  ...  2020-01-01 18:00:00  12.2  -7.3  25.0   
1    ONT      Ontario, CA  SFO  ...  2020-01-01 11:00:00   3.9  -3.4  59.0   
2    ONT      Ontario, CA  SJC  ...  2020-01-01 20:00:00  12.8  -7.3  24.0   

   prcp   wdir  wspd    pres coco target  
0   0.0  330.0  18.4  1017.3  3.0      1  
1   0.0  210.0   5.4  1017.6  2.0      0  
2   0.0  290.0  11.2  1015.8  2.0      0  

[3 rows x 29 columns]

In [5]:
flight_data['aircraft_type_new'] = [i.split('-')[0] for i in flight_data['Aircraft type']]

In [6]:
# list of numerical and categorical features to select 
l_numeric_cols = [ 'DEP_HOUR', 'temp', 'dwpt', 'rhum', 'prcp', 'wspd', 'pres', 'coco']
l_categorical_cols = ['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'aircraft_type_new']
target = "target"

In [7]:
# converting the categorical columns to categorical
for col in l_categorical_cols:
    flight_data[col] = flight_data[col].astype('category')

In [8]:
flight_data = flight_data[l_numeric_cols + l_categorical_cols + [target] + ['YEAR']]

In [9]:
train = flight_data[flight_data['YEAR'] == 2020]
test = flight_data[flight_data['YEAR'] == 2019]

In [10]:
del flight_data

In [11]:
parameters = {
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'num_class':4,
            "random_seed":42,
}

classifier = lgb.LGBMClassifier(**parameters)
classifier.fit(
    train.drop(["YEAR","target"], axis=1),
     train[target],
    categorical_feature=l_categorical_cols,
    verbose=1
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


LGBMClassifier(num_class=4, objective='multiclass', random_seed=42)

In [12]:
path = "./models"
filename = "2020train_LGBM.mdl"

In [13]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(classifier, f, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
# predicting the test results 
y_pred = classifier.predict(test.drop(["YEAR","target"], axis=1))

In [15]:
print(confusion_matrix(test[target],y_pred))

[[6497041      48     136    2858]
 [ 900736      67     237    2658]
 [ 313162      79     439    4263]
 [ 206771      97     502    8369]]


In [16]:
print(classification_report(test[target],y_pred))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90   6500083
           1       0.23      0.00      0.00    903698
           2       0.33      0.00      0.00    317943
           3       0.46      0.04      0.07    215739

    accuracy                           0.82   7937463
   macro avg       0.46      0.26      0.24   7937463
weighted avg       0.72      0.82      0.74   7937463



In [17]:
print("Testing  set score for model: %f" % classifier.score(test.drop(["YEAR","target"], axis=1)  , test[target] ))

Testing  set score for model: 0.819647


In [18]:
##### balanced class weight #####
parameters_b = {
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'num_class':4,
            "random_seed":42,
            "class_weight":"balanced"
}

classifier_b = lgb.LGBMClassifier(**parameters_b)
classifier_b.fit(
    train.drop(["YEAR","target"], axis=1),
     train[target],
    categorical_feature=l_categorical_cols,
    verbose=1
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


LGBMClassifier(class_weight='balanced', num_class=4, objective='multiclass',
               random_seed=42)

In [19]:
path = "./models"
filename = "2020train_LGBM_B.mdl"

In [20]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(classifier_b, f, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
# predicting the test results 
y_pred = classifier_b.predict(test.drop(["YEAR","target"], axis=1))

In [22]:
print(confusion_matrix(test[target],y_pred))

[[4272747  859598  580538  787200]
 [ 441396  181417  122401  158484]
 [ 126539   52097   56646   82661]
 [  72288   28606   35992   78853]]


In [23]:
print(classification_report(test[target],y_pred))

              precision    recall  f1-score   support

           0       0.87      0.66      0.75   6500083
           1       0.16      0.20      0.18    903698
           2       0.07      0.18      0.10    317943
           3       0.07      0.37      0.12    215739

    accuracy                           0.58   7937463
   macro avg       0.29      0.35      0.29   7937463
weighted avg       0.74      0.58      0.64   7937463



In [24]:
print("Testing  set score for model: %f" % classifier_b.score(test.drop(["YEAR","target"], axis=1)  , test[target] ))

Testing  set score for model: 0.578228
